In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns
plt.rcParams["figure.figsize"]=[15,5]

In [2]:
raw_sp = pd.read_csv('sp_jan.csv', usecols =['timestamp', 'data.A1'])

In [3]:
test_sp=raw_sp
sptimethresh=12

In [4]:
#Get boards from Screenprinter
test_sp=test_sp[['data.A1','timestamp']]
# events=scipy.signal.find_peaks(test_sp['data.A1'], height=(2.22, 5), width=1)
events_boards=scipy.signal.find_peaks(test_sp['data.A1'], height=(2.22, 5), distance=sptimethresh, width=1)
events_cleaning=scipy.signal.find_peaks(test_sp['data.A1'], height=9, distance=sptimethresh, width=1)


In [5]:
events_boards

(array([29066, 29142, 29453, 29474, 29510, 29620, 29653, 29665, 29734,
        29800, 29843, 29990, 30093, 31486, 31594, 31698, 31791, 31810,
        31899, 32009, 32111, 32213, 32311, 32414, 32429, 32522, 32625,
        32719, 32832, 32957, 33059, 33162, 33265, 33368, 33471, 33568,
        33671, 33779, 33882, 33985, 34083, 34098, 34186, 34294, 35630,
        35782, 35925, 35948, 36029, 36143, 36241, 36329, 36535, 36560,
        38223, 38899, 38921, 38952, 39060, 39107, 39147, 39193, 39276,
        39584, 39632, 39665, 39818, 39903, 40101, 40155, 40249, 40326,
        40688, 40710, 41259, 41368, 41421, 41442, 41554, 41586, 41620,
        41672, 41756, 41884, 41968, 42053, 42137, 42212, 42530, 42622,
        42696, 42778, 42793, 42866, 42952, 43036, 43301, 43387, 43468,
        43556, 43640, 43783, 43872, 43956, 44457, 44480, 44496, 44552,
        44573, 44594, 44617, 44677, 44762, 44817, 44859, 45032, 45075,
        45087, 45322, 45343, 45376, 45659, 45738, 45822, 45957, 46041,
      

In [20]:
# events_boards

In [6]:
import datetime as dt
import time
import matplotlib.dates as mdates

pattern="%Y-%m-%dT%H:%M:%S.%f"
detect_sp=test_sp[test_sp['state'] == 2]
test_sp['timestamp'] =  pd.to_datetime(test_sp['timestamp'], format=pattern)
detect_sp['timestamp'] =  pd.to_datetime(detect_sp['timestamp'], format=pattern)
detect_sp.head()

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,data.A1,timestamp,state
29631,15.31,2019-01-02 05:24:08.647470,2
29815,14.15,2019-01-02 05:28:11.501388,2
31503,13.25,2019-01-02 06:05:21.998008,2
31916,14.46,2019-01-02 06:14:28.045247,2
32328,13.44,2019-01-02 06:23:33.464741,2


In [7]:
# detect_sp=detect_sp[["timestamp"]].applymap(lambda x: x.seconds)

In [8]:
%matplotlib qt

In [9]:
fig, ax1 = plt.subplots()

ax1.stem(detect_sp['timestamp'], detect_sp['state'], 'red')

# ax1.xaxis.set_major_locator(mdates.DateFormatter('%H:%M'))

ax1.set_xlabel('Time')

ax2 = ax1.twinx()

ax2.plot(test_sp['timestamp'], test_sp['data.A1'], 'orange')

plt.title('Board Detections')

Text(0.5, 1.0, 'Board Detections')

In [10]:
test_sp['sum']=pd.Series.to_frame(test_sp['data.A1'].rolling(12, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
plt.plot(test_sp['timestamp'], test_sp['sum'])
plt.plot(test_sp['timestamp'], test_sp['data.A1']*10)

In [12]:
printing_delays_raw=scipy.signal.find_peaks(test_sp['sum'], height=(24, 30), distance=sptimethresh, width=1)[1]['widths']

In [13]:
import statistics

In [14]:
# printing_delays[1]['widths']
q1 = np.percentile(printing_delays_raw, 10)
q3 = np.percentile(printing_delays_raw, 95)
printing_delays = [i for i in printing_delays_raw if i > q1 and i < q3]
# printing_delays = filter(lambda x: x)

In [15]:
statistics.mean(printing_delays)
# printing_delays

12.992193390786397

In [16]:
sns.distplot(printing_delays,bins=10)

/home/raghu/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [17]:
cleaning_delays=scipy.signal.find_peaks(test_sp['sum'], height=60, distance=sptimethresh, width=1)[1]['widths']
# cleaning_delays

In [18]:
sns.distplot(cleaning_delays,bins=50)

In [19]:
statistics.mean(cleaning_delays)

12.350014895841353

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/raghu/anaconda3/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/raghu/anaconda3/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/raghu/anaconda3/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 346, in idle_draw
    self.draw()
  File "/home/raghu/anaconda3/lib/python3.6/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/raghu/anaconda3/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 402, in draw
    self.figure.draw(self.renderer)
  File "/home/raghu/anaconda3/lib/python3.6/site-packages/matplotlib/artist.py", line 50, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/raghu/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py", line 1649, in draw
    renderer, self, a